In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler, InputExample
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForMaskedLM, AutoConfig, BertConfig, AutoModelForCausalLM, BertGenerationDecoder
import pandas as pd
import json
import umap
from custom_TSDAE import *
import os
import umap
import matplotlib.pyplot as plt


# Load data

In [2]:
finetune_data = pd.read_csv("processed_data/clinvar_processed_snvs.csv")
finetune_data

,ReferenceSequence,MutatedSequence,ClinicalSignificance,OriginSimple,GeneSymbol,HGNC_ID,PhenotypeList,RS# (dbSNP)
0,CACCTGCGGGTGAGT,CACCTGCAGGTGAGT,Uncertain significance,germline,ZNF592,HGNC:28986,Galloway-Mowat syndrome 1,150829393
1,AAAGGTCCAGTCCTT,AAAGGTCTAGTCCTT,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606829
2,GTTGTCAACATGTAC,GTTGTCAGCATGTAC,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606830
3,GTCCCGAGGACTTCC,GTCCCGAAGACTTCC,Uncertain significance,germline,NUBPL,HGNC:20278,not provided|Inborn genetic diseases|Mitochond...,200401432
4,TATACGTGCCAGGTG,TATACGTACCAGGTG,Conflicting interpretations of pathogenicity; ...,germline,HFE,HGNC:4886,Hemochromatosis type 1|Porphyria cutanea tarda...,1800562
...,...,...,...,...,...,...,...,...
2507792,GAACCGGGACCTTGC,GAACCGGCACCTTGC,Likely pathogenic,germline,GAA,HGNC:4065,"Glycogen storage disease, type II",-1
2507793,CAGCAAGAGATGAGC,CAGCAAGTGATGAGC,Uncertain significance,germline,DHTKD1,HGNC:23537,Charcot-Marie-Tooth disease axonal type 2Q,778723208
2507794,GATGGGCCCAACGCC,GATGGGCTCAACGCC,Likely pathogenic,germline,COL2A1,HGNC:2200,Spondyloepiphyseal dysplasia congenita,-1
2507795,CTATGCCAACAAGCA,CTATGCCGACAAGCA,Likely pathogenic,germline,NAGLU,HGNC:7632,"Mucopolysaccharidosis, MPS-III-B",-1


In [3]:
test_data = pd.read_csv("processed_data/cogvig_data.csv")
test_data

,ReferenceSequence,MutatedSequence,type,Molecular_Consequence,Gene,disease,dbSNP
0,ACTATTCGGTGCGTT,ACTATTCTGTGCGTT,nonsynonymous SNV,SO:0001583|missense_variant\nSO:0001627|intron...,CDKN2A,Hereditary_cancer-predisposing_syndrome\nHered...,rs373407950(chr9-21974705-GtoT)
1,TTCCAAAATCCGTTT,TTCCAAAGTCCGTTT,nonsynonymous SNV,'-,TSC1,.,rs1374339026(chr9-135772897-AtoG)
2,CTTCGTTGCTTGCTG,CTTCGTTCCTTGCTG,nonsynonymous SNV,'-,RASA1,.,rs779313090(chr5-86564637-GtoC)
3,CTGCCACCCATCAGG,CTGCCACACATCAGG,nonsynonymous SNV,'-,CSF1R,.,rs753500118(chr5-149459663-CtoA)
4,TTAAATCTGGGAGGC,TTAAATCCGGGAGGC,nonsynonymous SNV,'-,RHOA,.,rs1310116062(chr3-49398430-TtoC)
...,...,...,...,...,...,...,...
418,CAAAACTGATAATCA,CAAAACTAATAATCA,stopgain,SO:0001587|nonsense,FANCC,"Fanconi_anemia\nFanconi_anemia,_complementatio...",rs121917784(chr9-98011537-GtoA)
419,ACCTGATCCATGTAA,ACCTGATTCATGTAA,nonsynonymous SNV,'-,PTCH1,.,rs780378700(chr9-98244421-CtoT)
420,AGTTCCATGGCCCTC,AGTTCCACGGCCCTC,nonsynonymous SNV,'-,PTCH1,.,rs755103500(chr9-98279102-TtoC)
421,GGATGAGAGTTACCT,GGATGAGCGTTACCT,nonsynonymous SNV,SO:0001583|missense_variant,DKC1,Dyskeratosis_congenita\nDyskeratosis_congenita...,rs146700772(chrX-153997508-AtoC)


# Instantiate model and dataloaders

In [4]:
model_name = 'zhihan1996/DNA_bert_3'
#model_name = "Peltarion/dnabert-minilm-mini"
tokenizer = AutoTokenizer.from_pretrained(model_name)

embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(768, 'cls')
model = SentenceTransformer(modules=[embedding_model, pooling_model])
model.tokenizer = tokenizer

decoder_config = BertConfig.from_json_file("decoder_config.json")

Some weights of the model checkpoint at zhihan1996/DNA_bert_3 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
train_dataset = CustomTSDAE_Dataset(finetune_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=64, collate_fn=lambda x: x, shuffle=True)

test_dataset = CustomTSDAE_Dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=lambda x: x, shuffle=False)

Preprocessing sequences
Preprocessing complete
Preprocessing sequences
Preprocessing complete


In [6]:
train_loss = CustomDenoisingAutoEncoderLoss(model, decoder_config, model_name)#, tie_encoder_decoder=True)

When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at zhihan1996/DNA_bert_3 were not used when initializing BertLMHeadModel: ['bert.encoder.layer.5.attention.self.key.weight', 'bert.encoder.layer.8.attention.output.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', 'bert.encoder.layer.3.attention.output.dense.weight', 'bert.encoder.layer.6.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.dense.bias', 'bert.encoder.layer.3.output.dense.bias', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.2.attention.self.key.bias', 'bert.encoder.layer.6.intermediate.dense.bias', 'bert.encoder.layer.9.output.LayerNorm.weight', 'bert.encoder.layer.11.output.LayerNorm.bias', 'bert.encoder.layer.5.attention.self.query.weight', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.output.dense.bias', 'bert.enco

In [7]:
dev_evaluator = LossEvaluator(test_dataloader, loss_model=train_loss, log_dir='logs/', name='dev')
best_loss = np.inf
if not os.path.exists("outputs"):
    os.mkdir("outputs")
    


def best_score_callback(score, epoch, steps):
    global best_loss
    
    if score<best_loss:
        print(f"New best model - Loss {score}")
        best_loss = score
        model.save('outputs/tsdae-model')
    return

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=20,
    steps_per_epoch=100,
    evaluator=dev_evaluator,
    weight_decay=1e-5,
    scheduler='constantlr',
    callback=best_score_callback,
    optimizer_params={'lr': 3e-6},
    show_progress_bar=True
)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

New best model - Loss 5.206889062557581


Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

New best model - Loss 4.1091347055615115


...

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

New best model - Loss 1.179368698372031


Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

New best model - Loss 1.1611525337651092


Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Load model with best validation loss
best_model = SentenceTransformer('outputs/tsdae-model')

# Embed COGVIC germline variants

In [ ]:
embedding_results = best_model(tokenizer(test_dataset.mutated_sequences, return_tensors="pt"))
variants_embeddings = embedding_results["sentence_embedding"].detach().numpy()
germline_embeddings = test_data[["dbSNP", "Gene", "disease"]].copy()
germline_embeddings.loc[:, [f"z{i}" for i in range(variants_embeddings.shape[-1])]] = variants_embeddings
germline_embeddings

In [ ]:
germline_embeddings.to_csv("COGVIC_germline_variants_embeddings.csv", index=False)

# Plot a representation of the cogvic embeddings

In [ ]:
reducer = umap.UMAP(n_components=2, random_state=42)
umap_embeddings = reducer.fit_transform(variants_embeddings)

fig, ax = plt.subplots(figsize=(8,8))

color = ["orange" if "Hereditary_cancer-predisposing_syndrome" in x else "blue" for x in germline_embeddings["disease"].values]
ax.scatter(umap_embeddings[:,0], umap_embeddings[:,1], c=color)
ax.set_xlabel("U1")
ax.set_ylabel("U2")

fig.savefig("images/COGVIC_UMAP_embeddings.png", facecolor="white")
plt.show()